In [2]:
def test_function():
    print("available functions")
    print("-preprocess_point_cloud(pcd, voxel_size,pprint_statements = False)")
    print("-prepare_dataset(source,target,voxel_size, trans_init = None,mytitle = "", print_statements = False)")
    print("-execute_global_registration(source_down, target_down, source_fpfh,target_fpfh, \nvoxel_size,print_statements = False)")
    print("-refine_registration(source, target,source_fpfh, target_fpfh,voxel_size,\nmytranformation = None,print_statements = False):")
    print("-stitch_two_point_clouds(source, target,mytitle,dt_string,voxel_size,\ncalculate_global = True,calculate_icp = True,trans_init = None,pprint_statements = False,save_statements = True,visualization_on = False)")
    print("-save_registration_result(source, target, transformation,title,save_result = True,visualize_result = False)")
    

In [ ]:
#%run "../Notebooks/Visualization_functions.ipynb"

In [5]:
# Functions

def visualize_tooth(t):
    geometry_list = [external_ply[t].paint_uniform_color([0.5, 0, 0]),
                     internal_ply[t].paint_uniform_color([0, 0.5, 0]),
                     upper_ply[t].paint_uniform_color([0, 0, 0.5])
                    ]
    o3d.visualization.draw_geometries(geometry_list,
                                      width=1000, height=800,
                                      window_name='3 raw views of tooth %s'%t
                                     )

def draw_registration_result(source, target, transformation, title = ""):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      width=1000, height=800,
                                      window_name='Open3D-'+str(title)
                                     )

def preprocess_point_cloud(pcd, voxel_size,pprint_statements = False):
    
    pcd_down = pcd.voxel_down_sample(voxel_size)
    radius_normal = voxel_size * 2
    radius_feature = voxel_size * 5
    
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=3))
    
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=10))
    
    if pprint_statements== True: 
        print("\n Downsample with a voxel size %.3f." % voxel_size)
        print("Estimate normal with search radius %.3f." % radius_normal)
        print("Compute FPFH feature with search radius %.3f." % radius_feature)

    return pcd_down, pcd_fpfh

def prepare_dataset(source,target,voxel_size,mytitle = "", print_statements = False):
    
    #if trans_init is not None:
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
        #trans_init = np.identity(4)
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4),title = mytitle )#np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size, pprint_statements = print_statements)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size, pprint_statements = print_statements)
    return source, target, source_down, target_down, source_fpfh, target_fpfh#,trans_init

#http://www.open3d.org/docs/release/tutorial/pipelines/global_registration.html

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size,
                                print_statements = False
                               ):
    distance_threshold = voxel_size *1.5
    
    if print_statements== True:
        print ("-"*50)
        print("global registration: RANSAC registration on downsampled point clouds.")
        #print("   Since the downsampling voxel size is %.3f," % voxel_size)
        #print("   we use a liberal distance threshold %.3f." % distance_threshold)
        
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.95),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(1000000, 0.999))
    
    #if print_statements== True: 
        #print ("type result: %s" %(type(result)))
    return result

#icp
def refine_registration(source, target, 
                        source_fpfh, target_fpfh, 
                        voxel_size,
                        mytranformation = None,
                        print_statements = False
                       ):
    
    distance_threshold = voxel_size #*5
    
    if print_statements== True: 
        print ("-"*100)
        print("Point-to-plane ICP registration is applied on original point")
        print("distance threshold %.3f." % distance_threshold)
        print("transformation used")
        print(mytranformation)
        
    #if type(mytranformation) != "numpy.ndarray":
        #mytranformation = np.identity(4)
        
    
    radius_normal = voxel_size * 2
    
    source.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=100))
    target.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=100))
    
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, mytranformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

def get_num_points(list_pointclouds, print_statement = False):
    num_points = []
    n_pc = list(range(len(list_pointclouds)))
    for i in n_pc:
        points = len(np.asarray(list_pointclouds[i].points))
        num_points.append(points)
    print("")
    if print_statement == True:
        print("")
        print("number of points in clouds")
        print (*zip(n_pc,num_points), sep = "\n")
    return num_points


def stitch_two_point_clouds(source, target, 
                            mytitle,dt_string, 
                            voxel_size,
                            preprocess_outliers = True,
                            calculate_global = True,
                            calculate_icp = True, # mark False if you want to skip this step
                            trans_init = None,
                            pprint_statements = False,
                            save_statements = True,
                            visualization_on = False,
                            final_vis_on = True,
                            myoverlapping_factor = 0.7,
                            maxnumattempts = 100,
                            myparams = None,
                            myconfiguration_file = None):
    #list of transformations
    trasformations_list = []
    
    # print 
    print ("stitching : %s" %mytitle)
    
    # Use different colors on the two point clouds
    source.paint_uniform_color([1, 0.706, 0])    #source is yellow
    target.paint_uniform_color([0, 0.651, 0.929])#target is blue
    
    # visualize
    geometry_list = [source,target]
    if visualization_on == True:
        o3d.visualization.draw_geometries(geometry_list,
                                          width=1000, height=800,
                                          window_name='Open3D-original %s'%mytitle)
    
    
    
    # outliers removal

    if preprocess_outliers == "statistical":
        
        my_nb_neighbors=1000
        my_std_ratio=0.001
        subtitle = "statistical outlier removal"
        if pprint_statements == True:
            print (subtitle)
        
        parameters = (my_nb_neighbors,my_std_ratio)
        parameters_labels = ("my_nb_neighbors","my_std_ratio")
        mytuples = list(zip(parameters_labels,parameters))

        processed_source, outlier_index_source = source.remove_statistical_outlier(nb_neighbors=my_nb_neighbors,
                                                                                    std_ratio=my_std_ratio)
        
        processed_target, outlier_index_target = target.remove_statistical_outlier(nb_neighbors=my_nb_neighbors,
                                                                                    std_ratio=my_std_ratio)
        
    elif preprocess_outliers == "no": 
        processed_source = source
        outlier_index_source = []
        processed_target = target
        outlier_index_target = []
        subtitle = "unprocessed source and target"
        if pprint_statements == True:
            print (subtitle)
        mytuples = ""

    else: # preprocess_outliers == "radius": # as default not to break previous code
        
        subtitle = "radius outlier removal"
        if pprint_statements == True:
            print (subtitle)
        source_nb_points= 25
        source_radius= 0.5
        target_nb_points= 25
        target_radius= 0.5
        
        parameters = (source_nb_points,
                      source_radius,
                      target_nb_points,
                      target_radius)
        
        parameters_labels = ("source_nb_points","source_radius","target_nb_points","target_radius")
        mytuples = list(zip(parameters_labels,parameters))
        
        processed_source, outlier_index_source = source.remove_radius_outlier(nb_points=source_nb_points,
                                                                              radius=source_radius)

        processed_target, outlier_index_target = target.remove_radius_outlier(nb_points=target_nb_points,
                                                                              radius=target_radius)
        
        
    # visualize without outliers      
    if visualization_on == True:
        #o3d.visualization.draw_geometries([processed_source,processed_target],
                                          #width=1000, height=800,
                                          #window_name='Open3D-processed %s'%mytitle)
        
        custom_draw_geometry_outliers(processed_source+processed_target, outlier_index_source+outlier_index_target, 
                          mytitle = subtitle, mytuples = mytuples,
                          params = myparams,  # parameter for camera point view, json file via pressing P
                          configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                          #fov_step  = 15,
                          rotate = True)
            

    # note: here we are re-defining the source and target as processed
    if pprint_statements == True:
        print ("dataset preparation")
    source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(processed_source,
                                                                                             processed_target,
                                                                                             voxel_size, 
                                                                                             #trans_init,
                                                                                             mytitle =mytitle,
                                                                                             print_statements = pprint_statements
                                                                                              )
    # append the just obtained one as trasformations_list
    #trasformations_list.append(trans_init)
    
    #-----------------------------------------------------------------------------------------------------
    #being in sequence the overlapping in a proper stitching should be high
    #Therefore we put a condition:
    #> while result_icp.correspondence_set < 0.7*(len(np.asarray(source.points))+len(np.asarray(target.points)))
    #> do execute_global_registration
    
    #points_source = len(np.asarray(source.points))
    #points_target = len(np.asarray(target.points))
    
    points_source,points_target = get_num_points([source,target],print_statement = pprint_statements)
    overlapping_points = np.zeros(1) # initialization 
    numattempts = 0
    
    while (numattempts <= maxnumattempts) and (len(np.asarray(overlapping_points)) < myoverlapping_factor*(points_target)):
    #-----------------------------------------------------------------------------------------------------    
        # ok in reality we always want to calculate the global ... or? 
        if calculate_global== True: 
            #global registration: execute ransac
            result_ransac = execute_global_registration(source_down, target_down,
                                                        source_fpfh, target_fpfh,
                                                        voxel_size,
                                                        print_statements = pprint_statements
                                                       )
            #print result
            if pprint_statements == True:
                print("Fit is:")
                print (type(result_ransac))
                print(result_ransac)
                print("Transformation is:")
                print(result_ransac.transformation)

            #save it
            if save_statements == True: 
                with open(dt_string+'-transformation_result_ransac-'+mytitle+'.pkl','wb') as f:
                    pkl.dump(result_ransac.transformation, f)
                if pprint_statements == True:
                    print("saved")

            threshold = 0.02

            if visualization_on == True: 
                draw_registration_result(source, target, 
                                         result_ransac.transformation,
                                         title = "%s global registration-%s"%(numattempts,mytitle),
                                        )

            # append the just obtained one as trasformations_list
            trasformations_list.append(result_ransac.transformation)

            points_source,points_target = get_num_points([source,target],print_statement = pprint_statements)
            overlapping_points = result_ransac.correspondence_set
            if pprint_statements == True:
                print ("overlapping points : " ,len(np.asarray(overlapping_points)))
                
            numattempts += 1
        #-----------------------------------------------------------------------------------------------------
        if calculate_icp== True: 
            #execute icp
            result_icp = refine_registration(source_down, target_down, 
                                             source_fpfh, target_fpfh,
                                             voxel_size,
                                             mytranformation =trasformations_list[-1],
                                             print_statements = pprint_statements
                                            )
            # print result
            if pprint_statements == True:
                #print("Fit is:")
                print(result_icp)
                print("Final Transformation is:")
                print(result_icp.transformation)
            #save it
            if save_statements == True: 
                with open(dt_string+'-transformation_result_icp-'+mytitle+'.pkl','wb') as f:
                    pkl.dump(result_icp.transformation, f)
                if pprint_statements == True:
                    print("saved")

            #append the just obtained one as trasformations_list
            trasformations_list.append(result_icp.transformation)


            points_source,points_target = get_num_points([source,target],print_statement = pprint_statements)
            overlapping_points = result_icp.correspondence_set
            if pprint_statements == True:
                print ("overlapping points : " ,len(np.asarray(overlapping_points)))
            
            if visualization_on == True: 
                draw_registration_result(source, target, 
                                         result_icp.transformation,
                                         title = "%s ICP registration-%s"%(numattempts,mytitle),
                                        )

        numattempts += 1
    #-----------------------------------------------------------------------------------------------------
    if final_vis_on == True: 
        draw_registration_result(source, target, 
                                 trasformations_list[-1],
                                 title = "last fit registration-%s"%mytitle
                                )
        

    #-----------------------------------------------------------------------------------------------------
    
    #save it
    newpointcloud = save_registration_result(source, target, trasformations_list[-1], 
                                             mytitle, 
                                             save_result = save_statements,
                                             visualize_result = False)
    
    return newpointcloud,trasformations_list

def save_registration_result(source, target, transformation,
                             title,
                             save_result = True,
                             visualize_result = False):
    
    filename = dt_string+'-stitch_'+title+'.pcd'
    
    # apply the chosen transformation to source and target
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    
    # combine them and create the newpoint cloud
    newpointcloud = source_temp + target_temp
    #newpointcloud.paint_uniform_color([0,0.5,0.1])
    
    #save
    if save_result == True: 
        o3d.io.write_point_cloud(filename, newpointcloud)
    
    #visualize
    if visualize_result == True:
        o3d.visualization.draw_geometries([newpointcloud],
                                          width=1000, height=800,
                                         window_name='newpointcloud-')
    return newpointcloud

